In [ ]:
import numpy as np
import random
import tensorflow as tf
from sklearn.datasets import make_blobs
from tensorflow.python.framework.random_seed import set_random_seed
from keras import initializers
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input
from keras.utils import to_categorical
from tensorflow.python.framework.random_seed import set_random_seed

X, y = make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)
trainX, valtestX, trainy, valtesty = train_test_split(X, y, train_size=0.1, shuffle=False)
valX, testX, valy, testy = train_test_split(valtestX, valtesty, train_size=0.112, shuffle=False)
trainy_cat, valy_cat, testy_cat = to_categorical(trainy), to_categorical(valy), to_categorical(testy)

print(trainX.shape, valX.shape, testX.shape)

random.seed(42)
np.random.seed(42)
set_random_seed(42)


In [ ]:
# Create the Neural Network model
nn_model = Sequential()
nn_model.add(Dense(25, input_dim=2, activation='relu'))
nn_model.add(Dense(3, activation='softmax'))

# Compile the model
nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = nn_model.fit(trainX, trainy_cat, epochs=500, validation_data=(valX, valy_cat), verbose=2)

# Evaluate the nn_model on the validation set
val_loss, val_accuracy = nn_model.evaluate(valX, valy_cat)


In [ ]:
_, train_acc = nn_model.evaluate(trainX, trainy_cat, verbose=0)
_, val_acc = nn_model.evaluate(valX, valy_cat, verbose=0)
_, test_acc = nn_model.evaluate(testX, testy_cat, verbose=0)
print('Train: %.3f, Val: %.3f, Test: %.3f' % (train_acc, val_acc, test_acc))
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='validation')
pyplot.legend()
pyplot.show()

In [ ]:
# Initialize Decision Tree model
dt_model = DecisionTreeClassifier(random_state=123, max_depth=2)

# Train the Decision Tree model on the training set
dt_model.fit(trainX, trainy)


In [ ]:
print('Train: %.3f, Val: %.3f, Test: %.3f' % (dt_model.score(trainX, trainy), dt_model.score(valX, valy), dt_model.score(testX, testy)))
nn_scores_train, nn_scores_val, nn_scores_test = (nn_model.predict(trainX), nn_model.predict(valX), nn_model.predict(testX))
dt_scores_train, dt_scores_val, dt_scores_test = (dt_model.predict_proba(trainX), dt_model.predict_proba(valX), dt_model.predict_proba(testX))


In [ ]:
# Create stacked datasets
stacked_X_train = np.hstack((nn_scores_train, dt_scores_train))
stacked_X_val = np.hstack((nn_scores_val, dt_scores_val))
stacked_X_test = np.hstack((nn_scores_test, dt_scores_test))

In [ ]:
print(stacked_X_train.shape, stacked_X_val.shape, stacked_X_test.shape)

In [ ]:
# Values of C to be tested
C_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

# Initialize variables to store the best accuracy and corresponding C value
best_accuracy = 0
best_C = None

# Train Logistic Regression models with different C values and select the best one
for C in C_values:
    # Initialize Logistic Regression model
    lr_model = LogisticRegression(max_iter=500, fit_intercept=True, C=C, random_state=123)

    # Train on the stacked training data
    lr_model.fit(stacked_X_train, trainy)

    # Predict on the validation set
    lr_val_predictions = lr_model.predict(stacked_X_val)

    # Calculate accuracy on the validation set
    accuracy = accuracy_score(valy, lr_val_predictions)

    # Update the best accuracy and corresponding C value
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C

# Train the best Logistic Regression model on the training set
best_lr_model = LogisticRegression(max_iter=500, fit_intercept=True, C=best_C, random_state=123)
best_lr_model.fit(stacked_X_train, trainy)

# Predict on the validation set and test set
lr_val_predictions = best_lr_model.predict(stacked_X_val)
lr_test_predictions = best_lr_model.predict(stacked_X_test)

# Calculate accuracy on the validation set and test set
val_accuracy = accuracy_score(valy, lr_val_predictions)
test_accuracy = accuracy_score(testy, lr_test_predictions)

# Print the accuracy of the best model on the validation and test sets
print(f'Best C value: {best_C}')
print(f'Accuracy on Validation Set: {val_accuracy}')
print(f'Accuracy on Test Set: {test_accuracy}')
